In [1]:
import numpy as np
import matplotlib.pyplot as plt
import field
import spatial
import timesteppers
import scipy.sparse.linalg as spla
from scipy import sparse

%matplotlib inline

### $ \frac{\partial p}{\partial t} = - \frac{\partial}{\partial x} [ \mu(x,t) p(x,t) ] + \frac{\partial^2}{\partial x^2} [ D(x,t) p(x,t) ]$

In [4]:
class FP_diffusion_split:
    
    def __init__(self, p, dD_p_dx2):

        self.grid = p.grid
        pt = Field(self.grid)
        
        LHS = pt - dD_p_dx2
        
        self.M = LHS.field_coeff(pt)
        self.L = LHS.field_coeff(p)
        
        self.F_ops = [0*p]        
    
class FP_advection_split:
    
    def __init__(self, p, dmu_p_dx):
        
        self.grid = p.grid
        pt = Field(self.grid)
        
        LHS = pt + dmu_p_dx
        
        self.M = LHS.field_coeff(pt)
        self.L = LHS.field_coeff(p)
        
        self.F_ops = [0*p]        
    
class FokkerPlanck_1D:
    
    def __init__(self, p, mu, D, spatial_order=4):
        '''
        p ~ Fields
        mu, D ~ Arrays
        '''
        
        self.grid = p.grid
        self.X = FieldSystem([p])

        pt = Field(self.grid)
         
        # generate derivatives
        dmu_p_dx = FiniteDifferenceUniformGrid(1, spatial_order, p*mu)
        dD_p_dx2 = FiniteDifferenceUniformGrid(2, spatial_order, p*D)
        
        
        self.operators = [FP_diffusion_split(self.p, dmu_p_dx),
                          FP_advection_split(self.p, dD_p_dx2)]
            
            
        self.ts_list   = [CrankNicolson(self.operators[0]),
                          PredictorCorrector(self.operators[1])]     
        
    def step(self, dt):
        # take a step
        # update self.t and self.iter
        
        # diffusive terms commute, only have to strang split the other terms
        self.ts_list[1].step(dt/2.)
        self.ts_list[0].step(dt)
        self.ts_list[1].step(dt/2.)
            
        
        # update self.t and self.iter
        self.t += dt
        self.iter += 1        